In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re 
import pyarabic.araby as araby
from nltk.corpus import stopwords
import nltk
from typing import List
from tqdm.notebook import tqdm
import arabic_reshaper 
from bidi.algorithm import get_display
import json 
import seaborn as sns

In [2]:
def data_loader(dir_path: str, n: int, read_metadata: bool = False, tags: str = "keywords", creator: str = "") -> pd.DataFrame:
    """
    Load episode transcripts from text files and optionally add metadata.

    :param dir_path: Path to the directory containing 'raw_data' and 'metadata' folders.
    :param n: Number of episodes to read.
    :param read_metadata: Whether to include metadata from JSON files.
    :return: A pandas DataFrame.
    """
    data_dir_path = os.path.join(dir_path, "raw_data")
    metadata_dir_path = os.path.join(dir_path, "metadata") if read_metadata else None

    episodes = []
    
    # Read the first `n` .txt files
    txt_files = [file for file in os.listdir(data_dir_path) if file.endswith(".txt")][:n]

    for file in txt_files:
        base_name = file.replace(".txt", "")
        file_path = os.path.join(data_dir_path, file)

        with open(file_path, 'r', encoding="utf-8") as f:
            episode_data = {"file_name": base_name, "episode_transcript": f.read().strip()}

        # Read metadata if needed
        if read_metadata:
            metadata_path = os.path.join(metadata_dir_path, base_name + ".json")
            if os.path.exists(metadata_path):
                with open(metadata_path, 'r', encoding="utf-8") as f:
                    json_file = json.load(f)
                    episode_data["length"] = json_file.get("length", None)
                    episode_data["tags"] = json_file.get(tags, None)
            else:
                episode_data["length"] = None  # Handle missing metadata
                episode_data["tags"] = None

        episodes.append(episode_data)
    df = pd.DataFrame(episodes)
    df['creator'] = creator
    return df

In [3]:
# Folders
DATA_FOLDERS = ["youtube/Al_Mokhbir_Al_Eqtisadi/raw_data", 'youtube/Da7ee7/raw_data', 'youtube/Fi_Al_Hadaraa/raw_data']
META_DATA = ["youtube/Al_Mokhbir_Al_Eqtisadi/metadata", 'youtube/Da7ee7/metadata', 'youtube/Fi_Al_Hadaraa/metadata']

n = 1000
MOKHBIR_DIR = "./youtube/Al_Mokhbir_Al_Eqtisadi/"
DA7EE7_DIR = "./youtube/Da7ee7/"
HADARAA_DIR = "./youtube/Fi_Al_Hadaraa/"
mokhbir_df = data_loader(MOKHBIR_DIR, n, read_metadata=True, creator="Al Mokhbir Al Eqtisadi")
mokhbir_df.head()

,file_name,episode_transcript,length,tags,creator
0,المخبر_الاقتصادي___كيف_أصبح_أصدقاء_إسرائيل...,عمان وقطر والبحرين والسعوديه دي هي الدول\nالعر...,00:22:52,"[اسرائيل, اشرف ابراهيم, البرلمان البريطاني, ال...",Al Mokhbir Al Eqtisadi
1,المخبر_الاقتصادي__كيف_ترفع_الشركات_أسعار_المن...,في اواخر 2016 حصل هجوم كبير من\nالمستهلكين في ...,00:15:12,"[أحجام المنتجات وأسعارها, أسعار المنتجات ترتفع...",Al Mokhbir Al Eqtisadi
2,المخبر_الاقتصادي__لماذا_تبيع_الصين_كميات_كبيرة...,في نوفمبر سنه 1971 وزراء ماليه مجموعه\nالدول ا...,00:16:53,"[أمريكا, أوروبا, اشرف ابراهيم, البنك المركزي ا...",Al Mokhbir Al Eqtisadi
3,المخبر_الاقتصادي__لماذا_سترتفع_أسعار_تذاكر_ال...,في شهر مايو اللي فات\n23 شركة طيران\nأعلنوا إن...,00:14:23,"[Airlines, fly, tickets, أسعار التذاكر, الخطوط...",Al Mokhbir Al Eqtisadi
4,المخبر_الاقتصادي__لماذا_تخفي_أمريكا_السلاح_ال...,في صباح يوم 19 اكتوبر 2023 الرئيس\nالامريكي جو...,00:20:25,"[AJ+, AJ+ الدحيح, AJ+ عربي, AJ+ كبريت, AJ+ مع ...",Al Mokhbir Al Eqtisadi


In [7]:
df = pd.read_csv('final_preprocessed_dataset.csv')
df.head()

,episode_transcript,creator,length,sarcasm,sentiment_score,cleaned_transcript,gemini_classifications,tf_idf_classifications,y
0,عمان وقطر والبحرين والسعوديه دي هي الدول\nالعر...,Al_Mokhbir_Al_Eqtisadi,00:22:52,غير ساخر,0.433861,عمان وقطر والبحرين والسعوديه الدول العربيه الا...,"امهات غزه, اطفال غزه, قتل الاطفال, رفح, الاحتلال","عينكم, خلوا, قصه","امهات غزه, اطفال غزه, قتل الاطفال, رفح, الاحتل..."
1,الراجل دا أنا عمري ما شفت حد زيه قبل كدا\nتحس ...,Al_Mokhbir_Al_Eqtisadi,00:14:45,غير ساخر,0.751848,الراجل عمري شفت زيه تحس عبارة مزيج خليط توماس ...,"العلاقات البريطانية الإسرائيلية, اللوبي الإسرا...","المحافظين, الاسرائيلي, حزب","العلاقات البريطانية الإسرائيلية, اللوبي الإسرا..."
2,في نوفمبر سنه 1971 وزراء ماليه مجموعه\nالدول ا...,Al_Mokhbir_Al_Eqtisadi,00:16:53,ساخر,0.006446,وزراء ماليه مجموعه الدول العشر الكبرى الجي اجت...,"الهجرة المغاربية إلى فرنسا, الاستعمار الفرنسي ...","المغاربة, المغاربية, فرنسا","الهجرة المغاربية إلى فرنسا, الاستعمار الفرنسي ..."
3,في اواخر شهر سبتمبر سنه 2023 وزير\nالماليه الن...,Al_Mokhbir_Al_Eqtisadi,00:24:44,ساخر,0.070738,اواخر وزير الماليه النيجيري اديبايو اووي ايدون...,"الحرب التكنولوجية بين أمريكا والصين, صناعة أشب...","الصين, دا, أشباه","الحرب التكنولوجية بين أمريكا والصين, صناعة أشب..."
4,في الساعات الاولى من صباح يوم 31 مايو\nسنه 201...,Al_Mokhbir_Al_Eqtisadi,00:14:11,غير ساخر,0.249777,الساعات وتحديدا الساعه فجرا قوات كوماندوز اسرا...,"جوهرة كوه نور, الاستعمار البريطاني للهند, شركة...","كوه, نور, الماسة","جوهرة كوه نور, الاستعمار البريطاني للهند, شركة..."


In [9]:
df['cleaned_transcript'].iloc[0]

'عمان وقطر والبحرين والسعوديه الدول العربيه الاربعه زارها ملك بريطانيا وامير ويلز تشارلز الثالث زوجته اميره ويلز الراحله الاميره ديانا تشارلز وديانا لفوا الدول العربيه الاربعه رحله مدتها الحقيقه الرحله اعتياديه الىى كبير كانش شيء مميز مثيل للاهتمام حواليها فجاه مرور الرحله رجعتي بقت محط اهتمام وسائل الاعلام البريطانيه صحيفه الديلي ميل البريطانيه لقت الارشيف العام رساله مؤرخه بتاريخ تشارلز باعتها لصديقه الكاتب الجنوب افريقي لورانس فاندر بوست الرساله تشارلز بيحكي للورانس زيارته للدول العربيه لسه خلصانه وبيشرح الزياره ساعدته يفهم افضل وجهه نظر العرب اسرائيل المهم المهم السياق تشارلس لصديقه بالنص رئيس امريكي عنده الشجاعه يقف وش اللوبي اليهودي الولايات المتحده ويواجهه وبعدين استدرك وكتب اعتقد ساذج افكر يحصل الرساله بمجرد خرجت للنور قلبت الدنيا تشارلز وقتها وريث العرش البريطاني وقدام الهجوم الكاسح انصار الاحتلال الاسرائيلي المتحدث القصر بتاعه طلع جماعه المكتوب الرساله اراء الامير تشارلز الشخصيه اراء الشخصيات قابلها زيارته للدول العربيه الحقيقه تنصل تشارلز كتبه رسالته متوقع كبير بريطانيا نعرف

In [18]:
from farasa.stemmer import FarasaStemmer

farasa_stemmer = FarasaStemmer(interactive=True)

def stem_text(text: str) -> str:
    return farasa_stemmer.stem(text)
text = df['cleaned_transcript'].iloc[0]
stem_text(text)

[2025-03-09 17:22:58,168 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


'عمان قطر بحرين سعودي دولة عربيه اربعه زار ملك بريطانيا أمير ويلز تشارلز ثالث زوجة أمير ويلز راحل اميره ديانا تشارلز وادي لف دولة عربيه اربعه رحل مدة حقيقه رحل اعتيادي الىى كبير كانش شيء مميز مثيل اهتمام حوالي فج مرور رحل رجعة قت محط اهتمام وسيلة اعلام بريطاني صحيف ديلي ميل بريطاني قت ارشيف عام رسال مؤرخ تاريخ تشارلز باعة صديق كاتب جنوب افريقي لورانس فاندر بوست رساله تشارلز حكى لورانس زيارة دولة عربيه لس خلص شرح زياره ساعد فهم أفضل وجه نظر عربي اسرائيل مهم مهم سياق تشارلس صديق نص رئيس امريكي عند شجاعه وقف وش لوبي يهودي ولاية المتحده واجه بعد استدرك كتب اعتقد ساذج افكر حصل رساله مجرد خرج نور قلب دنيا تشارلز وقت وريث عرش بريطاني قدام هجوم كاسح أنصار احتلال إسرائيلي متحدث قصر بتاع طلع جماع مكتوب رساله آراء امير تشارلز شخصيه آراء شخصية قابل زيارة دولة عربيه حقيقه تنصل تشارلز كتب رسالة متوقع كبير بريطانيا عرف صعب تسامح آراء نوع جايه مواطن بريطاني عادي مالوش حيث بال خرج شخص كل عارف هيبقى ملك بريطانيا مستقبلي وفى والد ملك إليزابيث تشارلز رسالة عمل مشكلة يعاير أمريكا لوبي مؤيد احتلال إسرائيلي 

In [19]:
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
def stem_text(text: str) -> str:
    return " ".join([st.stem(word) for word in text.split()])
text = df['cleaned_transcript'].iloc[0]
stem_text(text)

'عمن قطر بحر سعد دول عرب ربع زار ملك برط امر يلز شارلز ثلث زوج امر يلز رحل امر ديا شارلز ودا لفا دول عرب ربع رحل مدت حقق رحل اعتياديه لىى كبر كنش شيء ميز ثيل همم حلي فجه رور رحل رجع بقت محط همم وسل علم بريطانيه صحف ديل ميل بريطانيه لقت رشف عام رسل ؤرخ ارخ شارلز بعت صدق كتب جنب فرق رنس ندر بوس رسل شارلز بيح رنس زير دول عرب لسه خلص شرح زير سعد فهم فضل وجه نظر عرب رائيل مهم مهم سيق شارلس صدق لنص رئس امر عند شجع يقف وش لوب يهد ولي تحد يوج بعد درك كتب عقد سذج فكر حصل رسل جرد خرج نور قلب دنا شارلز وقت ورث عرش برط قدم هجم كسح صار حلل اسرائيلي تحدث قصر بتع طلع جمع كتب رسل ارء امر شارلز شخص ارء شخص قبل زير دول عرب حقق نصل شارلز كتب رسل توقع كبر برط عرف صعب سمح ارء نوع جيه وطن برط عدي الش حيث بلك خرج شخص لكل عرف هيبقى ملك برط قبل وفه دته لكه زابيث شارلز رسل عمل شكل عير امر لوب ؤيد حلل اسرائيلي يزل ؤثر سيس شرق وسط تلف ايم برط بقى كان وجد اقى لوب جمع ضغط ؤيد حلل اسرائيلي ورب غرب كله لوب اسرائيلي لند ؤثر وجب عمر رغم حدش كلم قرب ركز كله لوب وجد شنط اكد قوه فعل نسب حلل اسرائيلي لنه يثر سيس اقى دول عل